### **Tipos de cálculos:**

- ~~Rate of Change: P(hoy)/P(hoy – n días)-1~~

- ~~RSI (ver librería Análisis Técnico)~~

- ~~Moving average~~

- ~~Standard Deviation~~

- ~~Beta~~

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import app_functions as af
import plots as pl
import utils

import ta

## Params and data Loading

In [2]:
n_quantiles = 10
prices_fp = 'complete_prices.csv'
shift_period = 1
rets_period = 30

In [3]:
prices_df = pd.read_csv(prices_fp,index_col=0)
prices_df = prices_df.sort_index().fillna(limit=7,method='ffill')

In [4]:
monthly_filter = pd.read_csv(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\PriceClose_vertical\monthly_constituents_filter.csv',index_col=0)
daily_fiter_mask = pd.read_csv(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\PriceClose_vertical\daily_filter_mask.csv',index_col=0)

In [5]:
daily_dates = utils.create_dates_list('days',end_date=(2023,1,1),as_str=True)
monthly_dates = utils.create_dates_list('months',end_date=(2023,1,1),as_str=True)

In [6]:
monthly_prices = prices_df.loc[monthly_dates]

In [7]:
def do_all_monthly(df):
    masked_df = af.apply_mask(df,monthly_filter)
    ranked_df = af.rank_data(masked_df,10,'alto')
    rents_df = af.get_rents_df(ranked_df,monthly_prices,10,1,1)
    return rents_df

## Rate of change / Momentum
---

### 6 month momentum - Daily prices

In [30]:
def calculate_rate_of_change(df:pd.DataFrame,period:int):
    pct_change_df = df.pct_change(period,limit=1)
    return pct_change_df

In [31]:
pct_change_df = calculate_rate_of_change(prices_df,180)

In [32]:
monthly_masked_pct_change_df = af.apply_mask(pct_change_df,monthly_filter)

In [33]:
ranked_pct_change_df = af.rank_data(monthly_masked_pct_change_df,n_quantiles,'alto')
rents_pct_change_df = af.get_rents_df(ranked_pct_change_df,prices_df,n_quantiles,shift_period,rets_period)

In [34]:
pl.plot_NAV_absoluto(rents_pct_change_df)

In [35]:
pl.plot_sharpe(rents_pct_change_df)

### 6 month momentum - Monthly prices

In [36]:
momentum_6mo = prices_df.pct_change(180,limit=1)

In [37]:
done_momentum_6mo = do_all_monthly(momentum_6mo)

In [38]:
pl.plot_NAV_absoluto(done_momentum_6mo)

In [39]:
pl.plot_sharpe(done_momentum_6mo)

### Year momentum on monthly prices

In [40]:
monthly_pct_change = monthly_prices.pct_change(12,limit=1)

In [41]:
done_monthly_pct_change = do_all_monthly(monthly_pct_change)

In [42]:
pl.plot_NAV_absoluto(done_monthly_pct_change)

In [43]:
pl.plot_sharpe(done_monthly_pct_change)

## RSI
***

In [45]:
window = 14

In [46]:
def calculate_rsi(df:pd.DataFrame,window):
    rsi_df = df.apply(ta.momentum.rsi,axis=0,window=window,fillna=False)
    return rsi_df
rsi_df = calculate_rsi(prices_df,window)

In [47]:
done_rsi = do_all_monthly(rsi_df)

In [48]:
pl.plot_NAV_absoluto(done_rsi)

In [49]:
pl.plot_sharpe(done_rsi)

## Media Movil
***

In [50]:
rolling_window = 730

In [51]:
def calculate_simple_ma(df, rolling_window):
    sma_df = df.rolling(rolling_window).mean()
    return sma_df
sma_df = calculate_simple_ma(prices_df,rolling_window)

In [52]:
def calculate_exponential_ma(df:pd.DataFrame, rolling_window):
    ema_df = df.ewm(span=rolling_window).mean()
    return ema_df
ema_df = calculate_exponential_ma(prices_df,rolling_window)

In [53]:
done_sma = do_all_monthly(sma_df)

In [54]:
pl.plot_NAV_absoluto(done_sma)

In [55]:
done_ema = do_all_monthly(ema_df)

In [56]:
pl.plot_NAV_absoluto(done_ema)

In [57]:
pl.plot_sharpe(done_sma)

In [58]:
pl.plot_volatilidad(done_ema)

In [59]:
pl.plot_rentabilidad_media(done_ema)

## Beta
---

In [11]:
def calculate_beta(monthly_prices:pd.DataFrame, monthly_mask:pd.DataFrame, window_size_months:int):

    masked_monthly_prices = af.apply_mask(monthly_prices,monthly_mask) # should probably pass masked monthly prices directly instead of calculating inside the function
    market = masked_monthly_prices.mean(axis=1)

    window = window_size_months # Rolling window size in months
    
    df = monthly_prices.pct_change(limit=1) # Calculate monthly returns
    market_returns = market.pct_change(limit=1)

    covariance = df.rolling(window).cov(market_returns) # Calculate rolling covariance with market
    variance = market_returns.rolling(window).var() # Calculate rolling variance of market

    beta = covariance.div(variance,axis=0) # Calculate rolling beta
    return beta

In [32]:
beta = calculate_beta(monthly_prices,monthly_filter,12)

In [ ]:
done_beta = do_all_monthly(beta)

In [ ]:
pl.plot_NAV_absoluto(done_beta)

In [ ]:
pl.plot_rentabilidad_media(done_beta)

In [ ]:
pl.plot_volatilidad(done_beta)

## Standard deviation (rolling volatility)
---

In [ ]:
def calculate_rolling_std(df,window_size):

    std_df = df.rolling(window_size).std()
    return std_df

In [ ]:
std_df = calculate_rolling_std(prices_df,30)

In [ ]:
done_std = do_all_monthly(std_df)

In [ ]:
pl.plot_NAV_absoluto(done_std)

In [ ]:
pl.plot_sharpe(done_std)

In [ ]:
pl.plot_volatilidad(done_std)